In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
# instead of CNN, here we are using mobilenet. They are faster in process as compared to CNNs and they have less parameters.
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
# initialize the initial learning rate, number of epochs to train for and batch size.
# always keep the learnin rate less so loss will be calculated properly so better accuracy will be achieved soon
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

CATEGORIES = ["with_mask", "without_mask"]
data = []
labels = []

def get_data(data_dir):
    for category in CATEGORIES:
        path = os.path.join(data_dir, category)
        for img in os.listdir(path):
            img_path = os.path.join(path, img)
            image = load_img(img_path, target_size=(224, 224))
            image = img_to_array(image)
            image = preprocess_input(image)

            data.append(image)
            labels.append(category)

# perform one-hot encoding on the labels
# i.e text string "with mask" and "without mask" to 0 and 1
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

# construct the training image generator for data augmentation. Add many images from the same dataset by rotating, zooming, shifting, scaling, etc
aug =  ImageDataGenerator(
        rotation_range = 20,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        shear_range=0.15,
        horizontal_flip = True,  # randomly flip images
        fill_mode="nearest"
)

# load the MobileNetV2 network, ensuring the head Fully Connected layer sets are left off. include_top=False to keep fully connected layer at the top of the base model
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D((7, 7))(headModel)
headModel = Flatten(name = "Flatten")(headModel)
headModel = Dense(name = 128, activation = "relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(name = 2, activation = "softmax")(headModel) # output layer

# place the head Fully Connected model on top of the base model (this will become the actual model we will train)
model = Model(baseModel.input, outputs=headModel)

